# Upload the Order quantity from LE and preprocessing


In [1]:
import pandas as pd

In [2]:
pip install prophet

In [3]:
# change as needed
start_date = '2021-01-01'
end_date = '2023-12-01'
forecast_start = '2024-01-01'

In [4]:
orders = pd.read_csv("orders_cust.csv")

In [5]:
# Filter the orders
filtered_orders = orders[orders['Material'].astype(str).str.startswith('K') & ~orders['Material'].astype(str).str.startswith('KP')]

# Drop unnecessary columns
filtered_orders = filtered_orders.drop(columns=['D','Cl.','SPlt','Qty Requested.1','Setup','PO Quantity','Date','Fill Rate','Partial Fill Rate'])

# Convert dates to datetime
filtered_orders['Req.Date'] = pd.to_datetime(filtered_orders['Req.Date'])
filtered_orders['Deliv.dt'] = pd.to_datetime(filtered_orders['Deliv.dt'])

# Calculate date difference
filtered_orders['Date_Difference'] = (filtered_orders['Deliv.dt'] - filtered_orders['Req.Date']).dt.days

# Filter orders based on date difference
filtered_orders = filtered_orders[(filtered_orders['Date_Difference'] >= 0) & (filtered_orders['Date_Difference'] < 366)]

In [6]:
# Convert numeric columns
def convert_to_float(value):
    try:
        return float(value.replace(',', ''))
    except ValueError:
        return np.nan  # return NaN if the value cannot be converted

numeric_columns = ['Qty Requested']  # Add any other numeric columns as needed
for column in numeric_columns:
    filtered_orders[column] = filtered_orders[column].apply(convert_to_float).astype(float).astype(int)

# Drop additional unnecessary columns
filtered_orders = filtered_orders.drop(columns=['Item','Plnt','Deliv.dt','Release Dt','Date_Difference','PDT','IPT'])

In [7]:
# Group by 'Material' and 'Req.Date', and then sum the 'Qty Requested'
filtered_orders = filtered_orders.groupby(['Material', 'Req.Date'])['Qty Requested'].sum().reset_index()

In [8]:
filtered_orders['Material'].nunique()

1304

In [9]:
filtered_orders

,Material,Req.Date,Qty Requested
0,K1053-8,2018-01-02,7
1,K1053-8,2018-01-03,1
2,K1053-8,2018-01-04,1
3,K1053-8,2018-01-08,1
4,K1053-8,2018-01-09,4
...,...,...,...
128937,K96,2023-11-06,3
128938,K96,2023-11-07,3
128939,K96,2023-11-24,1
128940,K96,2023-12-05,2


# Introducing Product Segmentation


In [10]:
# Group by Material and calculate volume percentage and overall CV
grouped_df = filtered_orders.groupby('Material')['Qty Requested'].agg(['sum', 'mean', 'std'])
grouped_df['std'] = grouped_df['std'].fillna(0)

# Calculate the volume percentage
total_volume = grouped_df['sum'].sum()
grouped_df['Volume %'] = (grouped_df['sum'] / total_volume) * 100

# Calculate CV based on last year data or overall data if last year data is not available
latest_date = filtered_orders['Req.Date'].max()
one_year_ago = latest_date - pd.Timedelta(days=365)

last_year_orders = filtered_orders[filtered_orders['Req.Date'] >= one_year_ago]
last_year_grouped = last_year_orders.groupby('Material')['Qty Requested'].agg(['mean', 'std'])
last_year_grouped['std'] = last_year_grouped['std'].fillna(0)

grouped_df['CV'] = last_year_grouped['std'] / last_year_grouped['mean']
grouped_df['CV'] = grouped_df['CV'].fillna(grouped_df['std'] / grouped_df['mean'])

In [11]:
# Apply ranking
percentile_80 = grouped_df['sum'].quantile(0.8)
def rank_materials(row):
    if row['sum'] >= percentile_80:
        return 'Top 20%'
    else:
        return 'Below 80%'

grouped_df['Rank'] = grouped_df.apply(rank_materials, axis=1)

# Determine group based on CV and Rank
def determine_group(row):
    if row['CV'] <= 1:
        return 'Low Variance - High Volume' if row['Rank'] == 'Top 20%' else 'Low Variance - Low Volume'
    else:
        return 'High Variance - High Volume' if row['Rank'] == 'Top 20%' else 'High Variance - Low Volume'

grouped_df['Group'] = grouped_df.apply(determine_group, axis=1)


In [12]:
master_data = filtered_orders.merge(grouped_df['Group'],on='Material',how='left')
master_data = master_data.rename(columns={'Req.Date': 'Req.MonthYear'})

master_data.head()

,Material,Req.MonthYear,Qty Requested,Group
0,K1053-8,2018-01-02,7,High Variance - High Volume
1,K1053-8,2018-01-03,1,High Variance - High Volume
2,K1053-8,2018-01-04,1,High Variance - High Volume
3,K1053-8,2018-01-08,1,High Variance - High Volume
4,K1053-8,2018-01-09,4,High Variance - High Volume


In [13]:
#master_data.to_csv('Product_segments.csv')

In [14]:
print(master_data.groupby('Group')['Material'].nunique())

Group
High Variance - High Volume    128
High Variance - Low Volume     181
Low Variance - High Volume     133
Low Variance - Low Volume      862
Name: Material, dtype: int64


# Low Variance and Low Volume

In [15]:
df = master_data

#Select the materials having group Low Variance - Low Volume from the master data sheet

md1 = df[(df['Group']=='Low Variance - Low Volume') | (df['Group']=='High Variance - Low Volume') ]
md1['Material'].nunique()

md1 = md1[['Material','Req.MonthYear','Qty Requested']]
md1['Req.MonthYear'] = pd.to_datetime(md1['Req.MonthYear'],format='%m%Y')

md1 = md1[md1['Req.MonthYear'] >= '2018-01-01']

#Taking only 5 years for data from 2018 to 2023
df_filtered = md1[md1['Req.MonthYear'] >= '2018-01-01']
df_filtered['Material'].nunique()

1041

In [16]:
df_filtered

,Material,Req.MonthYear,Qty Requested
1103,K110,2018-02-16,1
1104,K110,2018-07-25,2
1105,K110,2018-09-10,1
1106,K110,2018-10-24,1
1107,K110,2019-01-11,8
...,...,...,...
127912,K939-1,2021-11-04,1
127913,K939-1,2022-01-03,1
127914,K939-1,2022-02-14,1
127915,K939-1,2023-08-10,1


In [17]:
import pandas as pd

df = df_filtered.copy()
df['Req.MonthYear'] = pd.to_datetime(df['Req.MonthYear']).dt.to_period('M').dt.to_timestamp()
df


,Material,Req.MonthYear,Qty Requested
1103,K110,2018-02-01,1
1104,K110,2018-07-01,2
1105,K110,2018-09-01,1
1106,K110,2018-10-01,1
1107,K110,2019-01-01,8
...,...,...,...
127912,K939-1,2021-11-01,1
127913,K939-1,2022-01-01,1
127914,K939-1,2022-02-01,1
127915,K939-1,2023-08-01,1


In [18]:
complete_df = pd.DataFrame()

# Process each material separately
for material in df['Material'].unique():
    # Filter the DataFrame for the current material
    temp_df = df[df['Material'] == material]

    # Group by 'Req.MonthYear' and sum 'Qty Requested'
    temp_df_grouped = temp_df.groupby('Req.MonthYear').agg({'Qty Requested': 'sum'}).reset_index()

    # Create a date range for the complete time series specific to this material
    if not temp_df_grouped.empty:
        date_range = pd.date_range(start=start_date,
                                   end=end_date,
                                   freq='MS')

        # Reindex the DataFrame to the full date range, filling missing months with zero
        temp_df_complete = temp_df_grouped.set_index('Req.MonthYear').reindex(date_range, fill_value=0).reset_index()
        temp_df_complete.columns = ['Req.MonthYear', 'Qty Requested']
        temp_df_complete['Material'] = material

        # Append to the complete DataFrame
        complete_df = pd.concat([complete_df, temp_df_complete], ignore_index=True)

# Display the completed DataFrame
(complete_df)

,Req.MonthYear,Qty Requested,Material
0,2021-01-01,1,K110
1,2021-02-01,0,K110
2,2021-03-01,1,K110
3,2021-04-01,0,K110
4,2021-05-01,0,K110
...,...,...,...
37471,2023-08-01,0,K960-4
37472,2023-09-01,0,K960-4
37473,2023-10-01,0,K960-4
37474,2023-11-01,0,K960-4


In [19]:
#Best fit model approach to find the Standard forecast
#Normal Code for the Best fit
import pandas as pd
import numpy as np
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from itertools import product

# Assuming complete_df is already loaded and has the correct format

complete_df['Req.MonthYear'] = pd.to_datetime(complete_df['Req.MonthYear'])

# Find the unique dates and sort them
unique_dates = np.sort(complete_df['Req.MonthYear'].unique())
#unique_dates.sort()

# Split the data into train and test based on the last three dates
test_dates = unique_dates[-3:]  # Last three dates for the test set
train = complete_df[~complete_df['Req.MonthYear'].isin(test_dates)]
test = complete_df[complete_df['Req.MonthYear'].isin(test_dates)]

# We will forecast each material separately
materials = complete_df['Material'].unique()

# Prepare a DataFrame to store the forecasts
forecast_df = test[['Req.MonthYear', 'Material']].drop_duplicates().reset_index(drop=True)
forecast_df['Prophet_Forecast'] = np.nan
forecast_df['HoltWinters_Forecast'] = np.nan
forecast_df['ARIMA_Forecast'] = np.nan

months_to_forecast = len(complete_df['Req.MonthYear'].unique())

for material in materials:
  material_train = train[train['Material'] == material]
  material_test = test[test['Material'] == material]

  # Prophet model
  prophet_df = material_train[['Req.MonthYear', 'Qty Requested']].rename(columns={'Req.MonthYear': 'ds', 'Qty Requested': 'y'})
  prophet_model = Prophet(interval_width=0.90)
  prophet_model.fit(prophet_df)
  future = prophet_model.make_future_dataframe(periods=len(material_test), freq='M')
  forecast_prophet = prophet_model.predict(future)

  # Holt-Winters model
  hw_model = ExponentialSmoothing(material_train['Qty Requested'], trend='add', seasonal='add', seasonal_periods=12)
  hw_model_fit = hw_model.fit()

  # Approximate standard error of the forecast
  rmse = np.sqrt(np.mean(hw_model_fit.resid ** 2))


  # ARIMA model
  arima_model = ARIMA(material_train['Qty Requested'], order=(1,1,1))
  arima_model_fit = arima_model.fit()
  forecast_arima = arima_model_fit.get_forecast(steps=len(material_test))
  forecast_arima_ci = forecast_arima.conf_int(alpha=0.1)


  # Assign forecasts to the corresponding material in forecast_df
  forecast_df.loc[forecast_df['Material'] == material, 'Prophet_Forecast'] = forecast_prophet['yhat'].values[-len(material_test):]
  forecast_df.loc[forecast_df['Material'] == material, 'HoltWinters_Forecast'] = (hw_model_fit.forecast(len(material_test)).values)
  forecast_df.loc[forecast_df['Material'] == material, 'ARIMA_Forecast'] = (arima_model_fit.forecast(len(material_test)).values)

  # Assign forecasts and CI to forecast_df
  forecast_df.loc[forecast_df['Material'] == material, 'Prophet_Forecast'] = forecast_prophet['yhat'].values[-len(material_test):]
  forecast_df.loc[forecast_df['Material'] == material, 'Prophet_Lower'] = forecast_prophet['yhat_lower'].values[-len(material_test):]
  forecast_df.loc[forecast_df['Material'] == material, 'Prophet_Upper'] = forecast_prophet['yhat_upper'].values[-len(material_test):]

  forecast_df.loc[forecast_df['Material'] == material, 'ARIMA_Forecast'] = forecast_arima.predicted_mean.values
  forecast_df.loc[forecast_df['Material'] == material, 'ARIMA_Lower'] = forecast_arima_ci.iloc[:, 0].values
  forecast_df.loc[forecast_df['Material'] == material, 'ARIMA_Upper'] = forecast_arima_ci.iloc[:, 1].values

  forecast_length = len(material_test)  # Number of periods to forecast
  forecast = hw_model_fit.forecast(steps=forecast_length)

    # Calculate the 90% confidence intervals
  confidence_level = 1.645  # Corresponds to 90%
  lower_bound = forecast - (confidence_level * rmse)
  upper_bound = forecast + (confidence_level * rmse)

  forecast_df.loc[forecast_df['Material'] == material, 'HW_Forecast'] = forecast.values
  forecast_df.loc[forecast_df['Material'] == material, 'HW_Lower'] = lower_bound.values
  forecast_df.loc[forecast_df['Material'] == material, 'HW_Upper'] = upper_bound.values



# Merge the forecasted values with the test data to compare
result_df = pd.merge(test, forecast_df, on=['Req.MonthYear', 'Material'], how='left')


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported in

In [20]:
#Future Prdiction for 3 months
import pandas as pd
import numpy as np
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from itertools import product

complete_df['Req.MonthYear'] = pd.to_datetime(complete_df['Req.MonthYear'])

# Find the unique dates and sort them
unique_dates = np.sort(complete_df['Req.MonthYear'].unique())


# Split the data into train and test based on the last three dates
test_dates = unique_dates[-3:]  # Last three dates for the test set
train = complete_df[~complete_df['Req.MonthYear'].isin(test_dates)]
test = complete_df[complete_df['Req.MonthYear'].isin(test_dates)]

materials = complete_df['Material'].unique()

extended_forecast_df = pd.DataFrame()

for material in materials:
    material_train = train[train['Material'] == material]
    material_test = test[test['Material'] == material]
    months_to_forecast = len(material_test['Req.MonthYear'].unique()) + 4

    # Prophet Forecasting
    prophet_df = material_train[['Req.MonthYear', 'Qty Requested']].rename(columns={'Req.MonthYear': 'ds', 'Qty Requested': 'y'})
    prophet_model = Prophet(interval_width=0.90)
    prophet_model.fit(prophet_df)
    future = prophet_model.make_future_dataframe(periods=months_to_forecast, freq='M')
    forecast_prophet = prophet_model.predict(future)

    # Holt-Winters Forecasting
    hw_model = ExponentialSmoothing(material_train['Qty Requested'], trend='add', seasonal='add', seasonal_periods=12).fit()
    forecast_hw = hw_model.forecast(months_to_forecast)

    # ARIMA Forecasting
    arima_model = ARIMA(material_train['Qty Requested'], order=(1,1,1)).fit()
    forecast_arima = arima_model.forecast(months_to_forecast)

    # Confidence intervals for ARIMA
    forecast_arima_ci = arima_model.get_forecast(steps=months_to_forecast).conf_int(alpha=0.1)  # 90% CI

    # Confidence intervals for Prophet
    prophet_lower = forecast_prophet['yhat_lower'][-months_to_forecast:]
    prophet_upper = forecast_prophet['yhat_upper'][-months_to_forecast:]

    # Confidence intervals for Holt-Winters
    rmse = np.sqrt(np.mean(hw_model.resid ** 2))
    confidence_level = 1.645  # Corresponds to 90%
    lower_bound = forecast_hw - (confidence_level * rmse)
    upper_bound = forecast_hw + (confidence_level * rmse)

    # Adding forecasts and confidence intervals to the DataFrame
    forecast_data = {
        'MonthYear': future['ds'][-months_to_forecast:],
        'Material': material,
        'Prophet_Forecast': forecast_prophet['yhat'][-months_to_forecast:],
        'Prophet_Lower': prophet_lower,
        'Prophet_Upper': prophet_upper,
        'HoltWinters_Forecast': forecast_hw,
        'HW_Lower': lower_bound,
        'HW_Upper': upper_bound,
        'ARIMA_Forecast': forecast_arima,
        'ARIMA_Lower': forecast_arima_ci.iloc[:, 0],
        'ARIMA_Upper': forecast_arima_ci.iloc[:, 1]
    }
    material_forecast_df = pd.DataFrame(forecast_data)
    extended_forecast_df = pd.concat([extended_forecast_df, material_forecast_df])

extended_forecast_df.reset_index(drop=True, inplace=True)


Streaming output truncated to the last 5000 lines.
INFO:cmdstanpy:Chain [1] done processing
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/

In [21]:
# # First, we'll filter the data from 2024 onwards
extended_forecast_df = extended_forecast_df[extended_forecast_df['MonthYear'] >= forecast_start]

# # Next, we rename the first column to 'Req.MonthYear'
extended_forecast_df.rename(columns={'MonthYear': 'Req.MonthYear'}, inplace=True)

# # Ensure that the dates are the first of every month
# # This will set all the dates to the first of the respective month
extended_forecast_df['Req.MonthYear'] = pd.to_datetime(extended_forecast_df['Req.MonthYear']).dt.to_period('M').dt.to_timestamp()

# # Display the first few rows to verify changes
# extended_forecast_df.head()
result_df = pd.concat([result_df,extended_forecast_df],axis=0)

<ipython-input-21-fa467d8dbb8b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extended_forecast_df.rename(columns={'MonthYear': 'Req.MonthYear'}, inplace=True)


In [22]:
# Assuming result_df is created by merging test and forecast_df dataframes
# Convert negative forecasts to 0 before calculating errors
for forecast_col in ['Prophet_Forecast', 'HoltWinters_Forecast', 'ARIMA_Forecast']:
    result_df[forecast_col] = result_df[forecast_col].apply(lambda x: max(x, 0))

# Now proceed with the error calculation as before
for forecast_col in ['Prophet_Forecast', 'HoltWinters_Forecast', 'ARIMA_Forecast']:
    result_df[forecast_col + '_Error'] = abs(result_df['Qty Requested'] - result_df[forecast_col])

# Apply the condition to adjust errors
result_df['Adjusted_Prophet_Error'] = np.where(result_df['Prophet_Forecast_Error'] > result_df['Qty Requested'], result_df['Qty Requested'], result_df['Prophet_Forecast_Error'])
result_df['Adjusted_HoltWinters_Error'] = np.where(result_df['HoltWinters_Forecast_Error'] > result_df['Qty Requested'], result_df['Qty Requested'], result_df['HoltWinters_Forecast_Error'])
result_df['Adjusted_ARIMA_Error'] = np.where(result_df['ARIMA_Forecast_Error'] > result_df['Qty Requested'], result_df['Qty Requested'], result_df['ARIMA_Forecast_Error'])

# Continue with grouping, summing, and calculating MAPE as before
material_sums = result_df.groupby('Material').agg({
    'Qty Requested': 'sum',
    'Adjusted_Prophet_Error': 'sum',
    'Adjusted_HoltWinters_Error': 'sum',
    'Adjusted_ARIMA_Error': 'sum',
}).reset_index()

#Calculate the MAPE
epsilon = 1e-8  # A small value to avoid division by zero
material_sums['Prophet_MAPE'] = material_sums['Adjusted_Prophet_Error'] / (material_sums['Qty Requested'] + epsilon)
material_sums['HoltWinters_MAPE'] = material_sums['Adjusted_HoltWinters_Error'] / (material_sums['Qty Requested'] + epsilon)
material_sums['ARIMA_MAPE'] = material_sums['Adjusted_ARIMA_Error'] / (material_sums['Qty Requested'] + epsilon)


# Merge the MAPE calculations back to the result_df
result_df = result_df.merge(material_sums[['Material', 'Prophet_MAPE', 'HoltWinters_MAPE', 'ARIMA_MAPE']], on='Material', how='left')




In [23]:
# ... (previous code for calculating MAPE values)

# Function to determine the model with the lowest MAPE
def get_best_model(row):
    mape_values = {
        'Prophet': row['Prophet_MAPE'],
        'HoltWinters': row['HoltWinters_MAPE'],
        'ARIMA': row['ARIMA_MAPE'],
    }
    return min(mape_values, key=mape_values.get)

# Apply the function to determine the best model for each material
material_sums['Best_Model'] = material_sums.apply(get_best_model, axis=1)

# Merge the Best_Model information back into the result_df
result_df = result_df.merge(material_sums[['Material', 'Best_Model']], on='Material', how='left')

# Keep only the forecast value and MAPE of the selected model
for index, row in result_df.iterrows():
    best_model = row['Best_Model']
    result_df.at[index, 'Selected_Forecast_Value'] = row[f'{best_model}_Forecast']
    result_df.at[index, 'Selected_MAPE'] = row[f'{best_model}_MAPE']


In [24]:
forecast_columns = ['Prophet_Forecast', 'HoltWinters_Forecast', 'ARIMA_Forecast']

result_df['Selected_MAPE'] = result_df.apply(lambda row: 0 if all(row[col] == 0 for col in forecast_columns) else row['Selected_MAPE'], axis=1)


In [ ]:
result_df

,Req.MonthYear,Qty Requested,Material,Prophet_Forecast,HoltWinters_Forecast,ARIMA_Forecast,Prophet_Lower,Prophet_Upper,ARIMA_Lower,ARIMA_Upper,...,ARIMA_Forecast_Error,Adjusted_Prophet_Error,Adjusted_HoltWinters_Error,Adjusted_ARIMA_Error,Prophet_MAPE,HoltWinters_MAPE,ARIMA_MAPE,Best_Model,Selected_Forecast_Value,Selected_MAPE
0,2023-10-01,0.0,K110,0.297901,0.000000,0.008668,0.058768,0.532238,-0.555421,0.572758,...,0.008668,0.0,0.0,0.0,0.0,0.0,0.0,Prophet,0.297901,0.0
1,2023-11-01,0.0,K110,0.409431,0.000000,0.005550,0.170489,0.645193,-0.563647,0.574748,...,0.005550,0.0,0.0,0.0,0.0,0.0,0.0,Prophet,0.409431,0.0
2,2023-12-01,0.0,K110,0.560385,0.000000,0.006672,0.319707,0.797936,-0.583035,0.596379,...,0.006672,0.0,0.0,0.0,0.0,0.0,0.0,Prophet,0.560385,0.0
3,2023-10-01,0.0,K119,0.000000,0.000000,0.391617,-17.626440,-9.746562,-6.469244,7.252478,...,0.391617,0.0,0.0,0.0,0.0,0.0,0.0,Prophet,0.000000,0.0
4,2023-11-01,0.0,K119,0.000000,0.000000,0.331797,-9.628079,-1.459189,-6.533060,7.196654,...,0.331797,0.0,0.0,0.0,0.0,0.0,0.0,Prophet,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6241,2024-02-01,NaN,K939-1,0.028223,0.325504,0.280097,-0.350111,0.417495,-0.456440,1.016635,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Prophet,0.028223,0.0
6242,2024-03-01,NaN,K939-1,0.000000,0.325655,0.280111,-0.430616,0.320662,-0.487983,1.048205,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0
6243,2024-01-01,NaN,K960-4,0.000000,0.000000,0.606125,-4.863044,4.350360,-5.206121,6.418370,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0
6244,2024-02-01,NaN,K960-4,0.000000,0.000000,0.606125,-4.871190,4.791317,-5.206121,6.418371,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0


# Max forecast for validation set

In [25]:
LE_Mape = 0.60
result_df['Selected_stddev'] = result_df.groupby('Material')['Selected_Forecast_Value'].transform('std')
mape_mean = result_df['Selected_MAPE'].mean()

In [26]:
MIF = (LE_Mape-mape_mean)/LE_Mape
MF = 1.65*(1-MIF)
result_df['Max_forecast'] = result_df['Selected_Forecast_Value'] + (MF*result_df['Selected_stddev'])

In [28]:
result_df['Max_factor'] = result_df['Max_forecast']/result_df['Selected_Forecast_Value']
mask = result_df['Max_factor'] > 2
# Replacing Max_forecast with Standard_forecast * 2 for those rows
result_df.loc[mask, 'Max_forecast'] = result_df['Selected_Forecast_Value'] * 2
final_df = result_df[['Req.MonthYear','Material','Qty Requested','Selected_Forecast_Value','Selected_MAPE','Max_forecast']]
result_df

,Req.MonthYear,Qty Requested,Material,Prophet_Forecast,HoltWinters_Forecast,ARIMA_Forecast,Prophet_Lower,Prophet_Upper,ARIMA_Lower,ARIMA_Upper,...,Adjusted_ARIMA_Error,Prophet_MAPE,HoltWinters_MAPE,ARIMA_MAPE,Best_Model,Selected_Forecast_Value,Selected_MAPE,Selected_stddev,Max_forecast,Max_factor
0,2023-10-01,0.0,K110,0.297901,0.000000,0.008669,0.053639,0.532816,-0.555420,0.572758,...,0.0,0.0,0.0,0.0,Prophet,0.297901,0.0,0.326465,0.409504,1.374631
1,2023-11-01,0.0,K110,0.409431,0.000000,0.005551,0.187469,0.650998,-0.563647,0.574749,...,0.0,0.0,0.0,0.0,Prophet,0.409431,0.0,0.326465,0.521034,1.272580
2,2023-12-01,0.0,K110,0.560385,0.000000,0.006673,0.302809,0.780430,-0.583033,0.596379,...,0.0,0.0,0.0,0.0,Prophet,0.560385,0.0,0.326465,0.671988,1.199154
3,2023-10-01,0.0,K119,0.000000,0.000000,0.391617,-17.223799,-9.661565,-6.469244,7.252478,...,0.0,0.0,0.0,0.0,Prophet,0.000000,0.0,4.157734,0.000000,NaN
4,2023-11-01,0.0,K119,0.000000,0.000000,0.331797,-9.432580,-1.720563,-6.533060,7.196654,...,0.0,0.0,0.0,0.0,Prophet,0.000000,0.0,4.157734,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6241,2024-02-01,NaN,K939-1,0.028223,0.325504,0.280079,-0.332288,0.361362,-0.456430,1.016588,...,NaN,0.0,0.0,0.0,Prophet,0.028223,0.0,0.759180,0.056446,2.000000
6242,2024-03-01,NaN,K939-1,0.000000,0.325655,0.280093,-0.446063,0.299051,-0.487969,1.048154,...,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0,0.759180,0.000000,NaN
6243,2024-01-01,NaN,K960-4,0.000000,0.000000,0.606125,-4.638833,4.150937,-5.206138,6.418387,...,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0,0.062065,0.000000,NaN
6244,2024-02-01,NaN,K960-4,0.000000,0.000000,0.606125,-4.765867,4.804978,-5.206138,6.418387,...,NaN,0.0,0.0,0.0,Prophet,0.000000,0.0,0.062065,0.000000,NaN


In [30]:
#result_df.to_csv('Final_low_vol_combined.csv')

In [ ]:
future_results = final_df[final_df['Req.MonthYear'] >= forecast_start]
future_results = future_results.drop(columns='Qty Requested')

In [ ]:
validation_results = final_df[final_df['Req.MonthYear'] < forecast_start]
validation_results = validation_results.drop(columns='Qty Requested')

In [ ]:
validation_results.to_csv('Validation_Low_vol.csv')

In [ ]:
future_results.to_csv('Future_Low_vol.csv')